In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

strava_client = stravalib.client.Client(access_token=access_token)
athlete = strava_client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [3]:
## Set up google sheets client, open worksheet
import pygsheets

gc = pygsheets.authorize(outh_file='client_secret.json', no_cache=True)

# Open spreadsheet and then workseet
sh = gc.open('Milburn Ultimate Scores')

In [4]:
## Get last entry from Raw Point Spreadsheet

wks = sh.worksheet_by_title('raw_points')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in wks.get_col(2) if d != '' and d != 'Start Time']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]

# lap_start_date = lap_start_date - timedelta(days=5)

lap_start_date

datetime.datetime(2018, 3, 20, 12, 54, 17)

In [5]:
runs = []
for activity in strava_client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        a = strava_client.get_activity(activity.id)
        runs.append(a)

In [6]:
def get_strava_description(activity, p=False):
    new_activity = strava_client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    try:
        color = color.lower()
    except AttributeError:
        color = None
        
    if p:
        print(new_activity.start_date)
        print(new_activity.description)
        print(team_score, opponent_score, color)
        
    return team_score, opponent_score, color

In [7]:
## Functions
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        try:
            lap_nums.append(int(l['name'].split(' ')[-1]))
            start_times.append(datetime.strptime(l['start_date_local'], '%Y-%m-%dT%H:%M:%SZ'))
            elapsed_times.append(timedelta(seconds=l['elapsed_time']))
        except TypeError:
            lap_nums.append(int(l.name.split(' ')[-1]))
            start_times.append(l.start_date_local)
            elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [8]:
## Compile raw points for export
games = []
all_data = []
for run in reversed(runs):
    team_wins, opponent_wins, color = get_strava_description(run, p=True)
    
    events = extract_events(run)

    point_df = pd.DataFrame(events, columns=['count', 'start_time', 'elapsed_time'])
    point_df['start_time'] = point_df['start_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    point_df['elapsed_time'] = point_df['elapsed_time'].apply(lambda x: x.seconds)
    
    data = point_df.sort_index(ascending=False).as_matrix().tolist()
    all_data = all_data + data
    
    
    

2018-03-21 17:11:45+00:00
1-2 colors
1 2 colors


In [9]:
## Write raw points to gsheets
wks = sh.worksheet_by_title('raw_points')
wks.insert_rows(2, values=all_data, number=len(all_data))

In [10]:
## Get last entry from Data Spreadsheet

wks = sh.worksheet_by_title('game_summaries')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in wks.get_col(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]
lap_start_date
# from datetime import timedelta
# lap_start_date = lap_start_date - timedelta(days=2)

datetime.datetime(2018, 3, 20, 0, 0)

In [11]:
wks = sh.worksheet_by_title('raw_points')
col_names = wks.get_all_values()[1][0:3]

val_lists = wks.get_all_values()[2:]
val_lists = [v[0:3] for v in val_lists]
processed_raw_points = pd.DataFrame(val_lists, columns=col_names)

# Process
processed_raw_points['Type'] = processed_raw_points['Type'].apply(lambda x: int(x))

processed_raw_points['Start Time'] = processed_raw_points['Start Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
processed_raw_points['Day'] = processed_raw_points['Start Time'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))
processed_raw_points['Elapsed Time (Sec)'] = processed_raw_points['Elapsed Time (Sec)'].apply(lambda x: timedelta(seconds=int(x)))


processed_raw_points = processed_raw_points[processed_raw_points['Start Time'] > lap_start_date]
processed_raw_points

,Type,Start Time,Elapsed Time (Sec),Day
0,2,2018-03-21 13:03:02,00:01:37,2018-03-21
1,2,2018-03-21 13:01:30,00:01:31,2018-03-21
2,1,2018-03-21 12:59:38,00:01:52,2018-03-21
3,2,2018-03-21 12:58:16,00:01:19,2018-03-21
4,3,2018-03-21 12:56:49,00:01:25,2018-03-21
5,2,2018-03-21 12:55:30,00:01:21,2018-03-21
6,1,2018-03-21 12:52:20,00:03:16,2018-03-21
7,2,2018-03-21 12:49:20,00:02:50,2018-03-21
8,1,2018-03-21 12:48:05,00:01:15,2018-03-21
9,4,2018-03-21 12:47:44,00:00:16,2018-03-21


In [12]:
for day, pdf in processed_raw_points.groupby('Day', sort=False):
    print(day)
    events = pdf[['Type', 'Start Time', 'Elapsed Time (Sec)']].as_matrix().tolist()
    
    for e in events:
        print(e)
    current_games = process_events(events)
    
    for g in current_games:
        g['my_color'] = color
        g['team_wins'] = team_wins
        g['opponent_wins'] = opponent_wins
        

    games = games + current_games


2018-03-21 00:00:00
[2, Timestamp('2018-03-21 13:03:02'), Timedelta('0 days 00:01:37')]
[2, Timestamp('2018-03-21 13:01:30'), Timedelta('0 days 00:01:31')]
[1, Timestamp('2018-03-21 12:59:38'), Timedelta('0 days 00:01:52')]
[2, Timestamp('2018-03-21 12:58:16'), Timedelta('0 days 00:01:19')]
[3, Timestamp('2018-03-21 12:56:49'), Timedelta('0 days 00:01:25')]
[2, Timestamp('2018-03-21 12:55:30'), Timedelta('0 days 00:01:21')]
[1, Timestamp('2018-03-21 12:52:20'), Timedelta('0 days 00:03:16')]
[2, Timestamp('2018-03-21 12:49:20'), Timedelta('0 days 00:02:50')]
[1, Timestamp('2018-03-21 12:48:05'), Timedelta('0 days 00:01:15')]
[4, Timestamp('2018-03-21 12:47:44'), Timedelta('0 days 00:00:16')]
[2, Timestamp('2018-03-21 12:47:42'), Timedelta('0 days 00:00:01')]
[1, Timestamp('2018-03-21 12:45:31'), Timedelta('0 days 00:02:11')]
[3, Timestamp('2018-03-21 12:43:54'), Timedelta('0 days 00:01:37')]
[2, Timestamp('2018-03-21 12:42:18'), Timedelta('0 days 00:01:35')]
[1, Timestamp('2018-03-21 12

In [13]:
df = pd.DataFrame(games).dropna()

In [14]:
df['date'] = df.end_time.apply(lambda x: date(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2018-03-21 0        2018-03-21 12:48:00   
           1        2018-03-21 12:31:32   
           2        2018-03-21 12:13:34   

                                                                events  \
date       game_num                                                      
2018-03-21 0         [(opponent_point, 2018-03-21 13:03:02, 0 days ...   
           1         [(opponent_point, 2018-03-21 12:47:42, 0 days ...   
           2         [(team_point, 2018-03-21 12:28:06, 0 days 00:0...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-21 0                0   colors         1               5   
           1                1   colors         1               5   
           2                2   colors         0               4   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-21 0                    2 2018-03-21 13:03:02           4         1   
           1                    2 2018-03-21 12:47:42           3         1   
           2                    2 2018-03-21 12:28:06           5         1   

                       win        date  
date       game_num                     
2018-03-21 0         False  2018-03-21  
           1         False  2018-03-21  
           2          True  2018-03-21

In [15]:
pdf = df

In [16]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [17]:
for (date, game_num), row in pdf.iterrows():      
    
    if row.my_color == 'white':
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'opponent_wins']            
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'opponent_wins']    
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [18]:
pdf

end_time  \
date       game_num                       
2018-03-21 0        2018-03-21 12:48:00   
           1        2018-03-21 12:31:32   
           2        2018-03-21 12:13:34   

                                                                events  \
date       game_num                                                      
2018-03-21 0         [(opponent_point, 2018-03-21 13:03:02, 0 days ...   
           1         [(opponent_point, 2018-03-21 12:47:42, 0 days ...   
           2         [(team_point, 2018-03-21 12:28:06, 0 days 00:0...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-21 0                0   colors         1               5   
           1                1   colors         1               5   
           2                2   colors         0               4   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-21 0                    2 2018-03-21 13:03:02           4         1   
           1                    2 2018-03-21 12:47:42           3         1   
           2                    2 2018-03-21 12:28:06           5         1   

                       win        date white_wins color_wins white_point  \
date       game_num                                                        
2018-03-21 0         False  2018-03-21          2          1           5   
           1         False  2018-03-21          2          1           5   
           2          True  2018-03-21          2          1           4   

                    color_point game_winner  
date       game_num                          
2018-03-21 0                  4       White  
           1                  3       White  
           2                  5       Color

In [19]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [28]:
scores = []
for (date, game_num), game in pdf.iterrows():
    base_dict = dict(date=date, end_time=game.end_time, game_num=game_num, white_wins=game.white_wins, color_wins=game.color_wins, game_winner=game.game_winner)    
    scores.append(merge_two_dicts(base_dict, dict(team='white', team_score=game.white_point, my_score=game.my_point if game.my_color == 'white' else None)))
    scores.append(merge_two_dicts(base_dict, dict(team='color', team_score=game.color_point, my_score=game.my_point if game.my_color == 'colors' else None)))

score_df = pd.DataFrame(scores).set_index(['date', 'game_num', 'team'], drop=False)

In [29]:
score_df

color_wins       date            end_time  game_num  \
date       game_num team                                                        
2018-03-21 0        white          1 2018-03-21 2018-03-21 12:48:00         0   
                    color          1 2018-03-21 2018-03-21 12:48:00         0   
           1        white          1 2018-03-21 2018-03-21 12:31:32         1   
                    color          1 2018-03-21 2018-03-21 12:31:32         1   
           2        white          1 2018-03-21 2018-03-21 12:13:34         2   
                    color          1 2018-03-21 2018-03-21 12:13:34         2   

                          game_winner  my_score   team  team_score white_wins  
date       game_num team                                                       
2018-03-21 0        white       White       NaN  white           5          2  
                    color       White       1.0  color           4          2  
           1        white       White       NaN  white           5          2  
                    color       White       1.0  color           3          2  
           2        white       Color       NaN  white           4          2  
                    color       Color       0.0  color           5          2

In [38]:
out_df = score_df[['date', 'white_wins', 'color_wins', 'game_num', 'game_winner', 'team', 'team_score', 'my_score']].fillna(value='')
out_df['date'] = out_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [39]:
out_df

date white_wins color_wins  game_num  \
date       game_num team                                                
2018-03-21 0        white  2018-03-21          2          1         0   
                    color  2018-03-21          2          1         0   
           1        white  2018-03-21          2          1         1   
                    color  2018-03-21          2          1         1   
           2        white  2018-03-21          2          1         2   
                    color  2018-03-21          2          1         2   

                          game_winner   team  team_score my_score  
date       game_num team                                           
2018-03-21 0        white       White  white           5           
                    color       White  color           4        1  
           1        white       White  white           5           
                    color       White  color           3        1  
           2        white       Color  white           4           
                    color       Color  color           5        0

In [40]:
data = out_df.as_matrix().tolist()
wks = sh.worksheet_by_title('game_summaries')
wks.insert_rows(2, values=data, number=len(data))